--- Day 5: If You Give A Seed A Fertilizer ---

You take the boat and find the gardener right where you were told he would be: managing a giant "garden" that looks more to you like a farm.

"A water source? Island Island is the water source!" You point out that Snow Island isn't receiving any water.

"Oh, we had to stop the water because we ran out of sand to filter it with! Can't make snow with dirty water. Don't worry, I'm sure we'll get more sand soon; we only turned off the water a few days... weeks... oh no." His face sinks into a look of horrified realization.

"I've been so busy making sure everyone here has food that I completely forgot to check why we stopped getting more sand! There's a ferry leaving soon that is headed over in that direction - it's much faster than your boat. Could you please go check it out?"

You barely have time to agree to this request when he brings up another. "While you wait for the ferry, maybe you can help us with our food production problem. The latest Island Island Almanac just arrived and we're having trouble making sense of it."

The almanac (your puzzle input) lists all of the seeds that need to be planted. It also lists what type of soil to use with each kind of seed, what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on. Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category - that is, soil 123 and fertilizer 123 aren't necessarily related to each other.

For example:

```
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
```

The almanac starts by listing which seeds need to be planted: seeds 79, 14, 55, and 13.

The rest of the almanac contains a list of maps which describe how to convert numbers from a source category into numbers in a destination category. That is, the section that starts with seed-to-soil map: describes how to convert a seed number (the source) to a soil number (the destination). This lets the gardener and his team know which soil to use with which seeds, which water to use with which fertilizer, and so on.

Rather than list every source number and its corresponding destination number one by one, the maps describe entire ranges of numbers that can be converted. Each line within a map contains three numbers: the destination range start, the source range start, and the range length.

Consider again the example seed-to-soil map:

50 98 2
52 50 48

The first line has a destination range start of 50, a source range start of 98, and a range length of 2. This line means that the source range starts at 98 and contains two values: 98 and 99. The destination range is the same length, but it starts at 50, so its two values are 50 and 51. With this information, you know that seed number 98 corresponds to soil number 50 and that seed number 99 corresponds to soil number 51.

The second line means that the source range starts at 50 and contains 48 values: 50, 51, ..., 96, 97. This corresponds to a destination range starting at 52 and also containing 48 values: 52, 53, ..., 98, 99. So, seed number 53 corresponds to soil number 55.

Any source numbers that aren't mapped correspond to the same destination number. So, seed number 10 corresponds to soil number 10.

So, the entire list of seed numbers and their corresponding soil numbers looks like this:

```
seed  soil
0     0
1     1
...   ...
48    48
49    49
50    52
51    53
...   ...
96    98
97    99
98    50
99    51
```

With this map, you can look up the soil number required for each initial seed number:

Seed number 79 corresponds to soil number 81. Seed number 14 corresponds to soil number 14. Seed number 55 corresponds to soil number 57. Seed number 13 corresponds to soil number 13. The gardener and his team want to get started as soon as possible, so they'd like to know the closest location that needs a seed. Using these maps, find the lowest location number that corresponds to any of the initial seeds. To do this, you'll need to convert each seed number through other categories until you can find its corresponding location number. In this example, the corresponding types are:

```
Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.
```

So, the lowest location number in this example is 35.

What is the lowest location number that corresponds to any of the initial seed numbers?

## Download Input
- https://adventofcode.com/2023/day/5/input
- select all, CMD+C
- open TextEdit, Make Plain Text
- CMD+V, save as input.txt

In [17]:
almanac_sample_map = '''seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4'''

In [11]:
# This function can be used for every puzzle to retrieve input
def get_input(filename="input.txt"):
    with open(filename, "r") as f:
        return f.read().strip()

# Let's save the input into the variable data
data = get_input()
# Split lines and make a list of strings
#data_lines = data.splitlines()
#data_lines[:10]

## Devise a Solution

- parsing strategy: split input using \n\n
- storing rules: dictionary where each key is the map name (like "seed-to-soil") and the value is a list of tuples or lists:
- Start with each seed number 
- Transform it through each stage using the rules
- Track what location each seed ends up at
- take the min of all locations

In [13]:
def get_rules(almanac_map):
    rules = {'seed2soil':[], 'soil2fertilizer':[], 'fertilizer2water':[], 'water2light':[], 'light2temp':[], 'temp2humidity':[], 'humidity2loc':[]}
    n = 0
    for key in rules:
        temp=almanac_map.split('\n\n')[n+1].split('map:\n')[1].splitlines()
        for i in range(len(temp)):
            temp[i] = tuple([int(x) for x in temp[i].split(' ')])
            rules[key] = temp
        n+=1    
    return rules
    
def apply_mapping(seed, rules):
    target = seed
    for key in rules:
        for i in rules[key]:
            dest_start = i[0]
            source_start = i[1]
            rang = i[2]
            if target in range(source_start,source_start+rang):
                target = dest_start+target-source_start
                break
            else:
                target = target
    loc = target
    return loc

def get_lowest_loc(input):
    seeds = input.splitlines()[0]
    seeds = seeds.split(': ')[1].split(' ')
    seeds = [int(i) for i in seeds]
    locs = []
    for seed in seeds:
        loc = apply_mapping(seed, rules = get_rules(input))
        locs.append(loc)
    return min(locs)

get_lowest_loc(data)



331445006

--- Part Two ---

Everyone will starve if you only plant such a small number of seeds. Re-reading the almanac, it looks like the seeds: line actually describes ranges of seed numbers.

The values on the initial seeds: line come in pairs. Within each pair, the first value is the start of the range and the second value is the length of the range. So, in the first line of the example above:
```
seeds: 79 14 55 13
```
This line describes two ranges of seed numbers to be planted in the garden. The first range starts with seed number 79 and contains 14 values: 79, 80, ..., 91, 92. The second range starts with seed number 55 and contains 13 values: 55, 56, ..., 66, 67.

Now, rather than considering four seed numbers, you need to consider a total of 27 seed numbers.

In the above example, the lowest location number can be obtained from seed number 82, which corresponds to soil 84, fertilizer 84, water 84, light 77, temperature 45, humidity 46, and location 46. So, the lowest location number is 46.

Consider all of the initial seed numbers listed in the ranges on the first line of the almanac. What is the lowest location number that corresponds to any of the initial seed numbers?

In [19]:
def apply_rule_to_range(range_start, range_length, rule):
    dest_start, source_start, rule_length = rule
    range_end = range_start + range_length
    source_end = source_start + rule_length
    
    # No overlap
    if range_end <= source_start or range_start >= source_end:
        return [], [(range_start, range_length)]
    
    # Calculate overlap
    overlap_start = max(range_start, source_start)
    overlap_end = min(range_end, source_end)
    
    mapped_ranges = []
    unmapped_ranges = []
    
    # Before overlap (unmapped)
    if range_start < overlap_start:
        unmapped_ranges.append((range_start, overlap_start - range_start))
    
    # Overlap (mapped)
    offset = dest_start - source_start
    mapped_ranges.append((overlap_start + offset, overlap_end - overlap_start))
    
    # After overlap (unmapped)
    if range_end > overlap_end:
        unmapped_ranges.append((overlap_end, range_end - overlap_end))
    
    return mapped_ranges, unmapped_ranges

def apply_mapping_stage(ranges, rules_list):
    result = []
    
    for range_start, range_length in ranges:
        current_ranges = [(range_start, range_length)]
        
        for rule in rules_list:
            next_ranges = []
            for r_start, r_len in current_ranges:
                mapped, unmapped = apply_rule_to_range(r_start, r_len, rule)
                result.extend(mapped)  # Mapped ranges are done
                next_ranges.extend(unmapped)  # Unmapped continue to next rule
            current_ranges = next_ranges
        
        # Any remaining unmapped ranges stay as-is
        result.extend(current_ranges)
    
    return result

def get_lowest_loc_part2(input):
    # Parse seeds as ranges
    seeds_line = input.splitlines()[0]
    seed_nums = [int(x) for x in seeds_line.split(': ')[1].split(' ')]
    
    # Create initial ranges (start, length)
    ranges = []
    for i in range(0, len(seed_nums), 2):
        ranges.append((seed_nums[i], seed_nums[i+1]))
    
    # Get all mapping rules
    rules = get_rules(input)
    
    # Apply each mapping stage in order
    stage_order = ['seed2soil', 'soil2fertilizer', 'fertilizer2water', 
                   'water2light', 'light2temp', 'temp2humidity', 'humidity2loc']
    
    for stage in stage_order:
        ranges = apply_mapping_stage(ranges, rules[stage])
    
    # Find minimum start value among all final ranges
    return min(r[0] for r in ranges)

# Test on sample
print("Sample:", get_lowest_loc_part2(almanac_sample_map))

# Run on real data
print("Real data:", get_lowest_loc_part2(data))

Sample: 46
Real data: 6472060
